In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Chronic_obstructive_pulmonary_disease_(COPD)"
cohort = "GSE32030"

# Input paths
in_trait_dir = "../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)"
in_cohort_dir = "../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)/GSE32030"

# Output paths
out_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/GSE32030.csv"
out_gene_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/gene_data/GSE32030.csv"
out_clinical_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/clinical_data/GSE32030.csv"
json_path = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Empirical Bayes Conditional Independence Graphs for Dense Regulatory Network Recovery"
!Series_summary	"Motivation: Computational inference methods that make use of graphical models to extract regulatory networks from gene expression data can have difficulty reconstructing dense regions of a network, a consequence of both computational complexity and unreliable parameter estimation when sample size is small. As a result, identification of hub genes is of special difficulty for these methods.Methods: We present a new algorithm, Empirical Light Mutual Min (ELMM), for large network reconstruction that has properties well suited for dense graph recovery. ELMM reconstructs the undirected graph of a regulatory network using empirical Bayes conditional independence testing with a heuristic relaxation of independence constraints in dense areas of the graph. This relaxation allows only one gene of a pair with a putative relation to be aware of the network 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import re
from typing import Dict, Any, Optional, Callable

# 1. Determine gene expression data availability
# Based on the background information, this dataset contains microarray data for gene expression
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (COPD status), it appears to be in row 2
trait_row = 2

# Age data doesn't appear to be available in the sample characteristics
age_row = None

# Gender data doesn't appear to be available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    if pd.isna(value):
        return None
    
    # Extract the value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    # Check for COPD status
    if "yes" in value:
        return 1
    elif "no" in value:
        return 0
    return None

def convert_age(value):
    # This function isn't needed since age data isn't available
    return None

def convert_gender(value):
    # This function isn't needed since gender data isn't available
    return None

# 3. Save Metadata
# Check if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None

# Validate and save initial cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create clinical data DataFrame
    # The sample characteristics dictionary has rows as keys and values as lists
    sample_char_dict = {
        0: ['department of genetic medicine id: DGM-00028', 'ethnicity: Afr', 'department of genetic medicine id: DGM-00060', 'department of genetic medicine id: DGM-00061', 'department of genetic medicine id: DGM-00104', 'department of genetic medicine id: DGM-10131', 'department of genetic medicine id: DGM-00167', 'department of genetic medicine id: DGM-00171', 'department of genetic medicine id: DGM-00262', 'department of genetic medicine id: DGM-00313', 'department of genetic medicine id: DGM-10287', 'department of genetic medicine id: DGM-00382', 'department of genetic medicine id: DGM-10139', 'department of genetic medicine id: DGM-00471', 'department of genetic medicine id: DGM-00512', 'department of genetic medicine id: DGM-02088', 'department of genetic medicine id: DGM-10152', 'department of genetic medicine id: DGM-00632', 'department of genetic medicine id: DGM-00683', 'department of genetic medicine id: DGM-10404', 'department of genetic medicine id: DGM-00791', 'department of genetic medicine id: DGM-10132', 'department of genetic medicine id: DGM-10135', 'department of genetic medicine id: DGM-01275', 'department of genetic medicine id: DGM-10144', 'department of genetic medicine id: DGM-10130', 'department of genetic medicine id: DGM-10406', 'department of genetic medicine id: DGM-01607', 'department of genetic medicine id: DGM-01737', 'department of genetic medicine id: DGM-01749'],
        1: ['smoking status: S', 'department of genetic medicine id: DGM-00038', 'department of genetic medicine id: DGM-00029', 'department of genetic medicine id: DGM-00030', 'department of genetic medicine id: DGM-00035', 'department of genetic medicine id: DGM-00036', 'department of genetic medicine id: DGM-00041', 'department of genetic medicine id: DGM-00044', 'department of genetic medicine id: DGM-00052', 'department of genetic medicine id: DGM-00073', 'department of genetic medicine id: DGM-00069', 'department of genetic medicine id: DGM-00072', 'department of genetic medicine id: DGM-00078', 'smoking status: NS', 'department of genetic medicine id: DGM-00123', 'department of genetic medicine id: DGM-00244', 'department of genetic medicine id: DGM-00249', 'department of genetic medicine id: DGM-00416', 'department of genetic medicine id: DGM-00465', 'department of genetic medicine id: DGM-00530', 'department of genetic medicine id: DGM-00548', 'department of genetic medicine id: DGM-00551', 'department of genetic medicine id: DGM-00553', 'department of genetic medicine id: DGM-00625', 'department of genetic medicine id: DGM-00661', 'department of genetic medicine id: DGM-00685', 'department of genetic medicine id: DGM-00696', 'department of genetic medicine id: DGM-00757', 'department of genetic medicine id: DGM-00751', 'department of genetic medicine id: DGM-00778'],
        2: ['copd status: yes', 'smoking status: S', 'smoking status: NS', np.nan, 'serum 25-oh-d: low vitamin D', 'serum 25-oh-d: mid vitamin D', 'serum 25-oh-d: high vitamin D', 'cell type: mixture of epithelium, basal, ciliated, secretory, undifferentiated and inflammatory cells'],
        3: [np.nan, 'copd status: yes', 'serum 25-oh-d: mid vitamin D', 'serum 25-oh-d: high vitamin D', 'serum 25-oh-d: low vitamin D', 'time: n/a']
    }
    
    # Using the correct format for the geo_select_clinical_features function
    clinical_data = pd.DataFrame.from_dict(sample_char_dict, orient='index')
    
    # Extract clinical features using the library function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the DataFrame to check extraction results
    preview_results = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview_results)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{0: [1.0], 1: [nan], 2: [nan], 3: [nan], 4: [nan], 5: [nan], 6: [nan], 7: [nan], 8: [nan], 9: [nan], 10: [nan], 11: [nan], 12: [nan], 13: [nan], 14: [nan], 15: [nan], 16: [nan], 17: [nan], 18: [nan], 19: [nan], 20: [nan], 21: [nan], 22: [nan], 23: [nan], 24: [nan], 25: [nan], 26: [nan], 27: [nan], 28: [nan], 29: [nan]}
Clinical data saved to ../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/clinical_data/GSE32030.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)/GSE32030/GSE32030_series_matrix.txt.gz


Gene data shape: (54675, 270)
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Review gene identifiers
# The gene identifiers (e.g., '1007_s_at', '1053_at') appear to be Affymetrix probe IDs
# rather than standard human gene symbols (which would look like BRCA1, TP53, etc.)
# These need to be mapped to gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns in gene_annotation contain probe IDs and gene symbols
# Based on the preview, 'ID' contains probe IDs and 'Gene Symbol' contains gene symbols
id_column = 'ID'
gene_symbol_column = 'Gene Symbol'

# 2. Get a gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, id_column, gene_symbol_column)
print(f"Gene mapping shape: {gene_mapping.shape}")
print("Preview of gene mapping dataframe:")
print(preview_df(gene_mapping))

# 3. Apply gene mapping to convert from probe-level to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("Preview of gene expression data after mapping:")
print(preview_df(gene_data))

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the gene data to CSV
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping shape: (45782, 2)
Preview of gene mapping dataframe:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1 /// MIR4640', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}


Gene expression data shape after mapping: (21278, 270)
Preview of gene expression data after mapping:
{'GSM549645': [0.0, 0.0, 0.0, 0.0, 0.0], 'GSM549646': [83.487, 24.5968, 191.55792000000002, 2417.498, 155.804], 'GSM549647': [0.0, 0.0, 0.0, 0.0, 0.0], 'GSM549648': [0.0, 0.0, 0.0, 0.0, 0.0], 'GSM549649': [0.0, 0.0, 0.0, 0.0, 0.0], 'GSM549650': [0.0, 0.0, 0.0, 0.0, 0.0], 'GSM549651': [0.0, 0.0, 0.0, 0.0, 0.0], 'GSM549652': [0.0, 0.0, 0.0, 0.0, 0.0], 'GSM549653': [0.0, 0.0, 0.0, 0.0, 0.0], 'GSM549654': [0.0, 0.0, 0.0, 0.0, 0.0], 'GSM549655': [0.0, 0.0, 0.0, 0.0, 0.0], 'GSM549656': [0.0, 0.0, 0.0, 0.0, 0.0], 'GSM549657': [0.0, 0.0, 0.0, 0.0, 0.0], 'GSM549658': [0.0, 0.0, 0.0, 0.0, 0.0], 'GSM549659': [0.0, 0.0, 0.0, 0.0, 0.0], 'GSM549660': [0.0, 0.0, 0.0, 0.0, 0.0], 'GSM549661': [0.0, 0.0, 0.0, 0.0, 0.0], 'GSM549662': [0.0, 0.0, 0.0, 0.0, 0.0], 'GSM549663': [0.0, 0.0, 0.0, 0.0, 0.0], 'GSM549664': [0.0, 0.0, 0.0, 0.0, 0.0], 'GSM549665': [0.0, 0.0, 0.0, 0.0, 0.0], 'GSM549666': [0.0, 0.0, 0.

Gene expression data saved to ../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/gene_data/GSE32030.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data 
# (This was already done in the previous step, so no need to repeat)
print(f"Normalized gene data shape: {gene_data.shape}")

# 2. Examine the sample IDs in the gene expression data to understand the structure
print("Gene data column names (sample IDs):")
print(gene_data.columns[:5])  # Print first 5 for brevity

# Inspect the clinical data format from the matrix file directly
background_info, clinical_data = get_background_and_clinical_data(matrix_file)
print("\nRaw clinical data structure:")
print(f"Clinical data shape: {clinical_data.shape}")
print(f"Clinical data columns: {clinical_data.columns[:5]}")

# Get the sample characteristics to re-extract the disease information
sample_characteristics_dict = get_unique_values_by_row(clinical_data)
print("\nSample characteristics dictionary:")
print(sample_characteristics_dict)

# 3. Directly create clinical features from the raw data again
# Verify trait row contains the disease information (OA vs RA)
print("\nValues in trait row:")
trait_values = clinical_data.iloc[trait_row].values
print(trait_values[:5])

# Create clinical dataframe with proper structure
# First get the sample IDs from gene data as these are our actual sample identifiers
sample_ids = gene_data.columns.tolist()

# Create the clinical features dataframe with those sample IDs
clinical_features = pd.DataFrame(index=[trait], columns=sample_ids)

# Fill the clinical features with our trait values by mapping GSM IDs to actual values
for col in clinical_data.columns:
    if col in sample_ids:
        # Extract the disease value and convert it
        disease_val = clinical_data.iloc[trait_row][col]
        clinical_features.loc[trait, col] = convert_trait(disease_val)

print("\nCreated clinical features dataframe:")
print(f"Shape: {clinical_features.shape}")
print(clinical_features.iloc[:, :5])  # Show first 5 columns

# 4. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
print(f"\nLinked data shape before handling missing values: {linked_data.shape}")

# 5. Handle missing values - we need to use the actual column name, not the trait variable
# First identify the actual trait column name in the linked data
trait_column = clinical_features.index[0]  # This should be 'Osteoarthritis'
print(f"Actual trait column in linked data: {trait_column}")

# Now handle missing values with the correct column name
linked_data_clean = handle_missing_values(linked_data, trait_column)
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# 6. Evaluate bias in trait and demographic features
is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait_column)

# 7. Conduct final quality validation
note = "Dataset contains gene expression data from synovial fibroblasts of RA and OA patients. Data includes high serum and low serum responses."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=(linked_data_clean.shape[0] > 0),
    is_biased=is_biased,
    df=linked_data_clean,
    note=note
)

# 8. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable due to quality issues - linked data not saved")

Normalized gene data shape: (21278, 270)
Gene data column names (sample IDs):
Index(['GSM549645', 'GSM549646', 'GSM549647', 'GSM549648', 'GSM549649'], dtype='object')



Raw clinical data structure:
Clinical data shape: (4, 271)
Clinical data columns: Index(['!Sample_geo_accession', 'GSM549645', 'GSM549646', 'GSM549647',
       'GSM549648'],
      dtype='object')

Sample characteristics dictionary:
{0: ['department of genetic medicine id: DGM-00028', 'ethnicity: Afr', 'department of genetic medicine id: DGM-00060', 'department of genetic medicine id: DGM-00061', 'department of genetic medicine id: DGM-00104', 'department of genetic medicine id: DGM-10131', 'department of genetic medicine id: DGM-00167', 'department of genetic medicine id: DGM-00171', 'department of genetic medicine id: DGM-00262', 'department of genetic medicine id: DGM-00313', 'department of genetic medicine id: DGM-10287', 'department of genetic medicine id: DGM-00382', 'department of genetic medicine id: DGM-10139', 'department of genetic medicine id: DGM-00471', 'department of genetic medicine id: DGM-00512', 'department of genetic medicine id: DGM-02088', 'department of genetic m

/media/techt/DATA/GenoAgent/tools/preprocess.py:400: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  linked_data = pd.concat([clinical_df, genetic_df], axis=0).T


/media/techt/DATA/GenoAgent/tools/preprocess.py:455: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[gene_cols] = df[gene_cols].fillna(df[gene_cols].mean())


Linked data shape after handling missing values: (35, 21279)
Quartiles for 'Chronic_obstructive_pulmonary_disease_(COPD)':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1
Max: 1
The distribution of the feature 'Chronic_obstructive_pulmonary_disease_(COPD)' in this dataset is severely biased.

Dataset deemed not usable due to quality issues - linked data not saved
